In [56]:
import torch

In [57]:
maping = lambda x: torch.exp(-x)

In [58]:
X = torch.rand((100, 2))
Y = torch.sum(maping(X), dim=1)

In [59]:
X.shape, Y.shape

(torch.Size([100, 2]), torch.Size([100]))